In [21]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import nltk
import swifter
from tqdm.notebook import tqdm
tqdm.pandas()
import pickle

df = pd.read_csv(f'Download/Cleaned Data/Confession_cleaned.csv')
df = df.loc[df['created_utc']>1546300800].copy()



sbert = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

df['sent_count'] = df.swifter.progress_bar(enable=True, desc='Counting sentences').apply(lambda row: len(nltk.sent_tokenize(row['selftext'])), axis=1)

df = df[~(df['sent_count']==0)]

df['sbert_sum'] = df.progress_apply(lambda row: sum([sbert.encode(sent) for sent in nltk.sent_tokenize(row['selftext'])]), axis=1)
df['sbert_avg'] = df.progress_apply(lambda row: row['sbert_sum']/row['sent_count'], axis=1)

pickle.dump(df, open(f'Download/Cleaned Data/Confession.pickle', 'wb'))

  0%|          | 0/88343 [00:00<?, ?it/s]

  0%|          | 0/88343 [00:00<?, ?it/s]

In [24]:
import pickle
pickle.load(open('Download/Cleaned Data/NoSleep.pickle', 'rb'))

,index,subreddit,id,created_utc,title,selftext,sent_count,sbert_sum,sbert_avg
138696,0,nosleep,abcv3p,1546300904,We Wanted A Killer To Join Our New Year’s Eve ...,We'd gotten together for New Year's Eve. Only ...,592,"[34.271263, -28.094067, 142.04535, -169.51334,...","[0.057890646, -0.047456194, 0.23994146, -0.286..."
138697,1,nosleep,abd1fa,1546301994,I Met The Devil in the Desert of Nevada,"First, I got lost while I was adventuring in t...",65,"[-6.0869465, -4.4725637, 11.303014, -16.207602...","[-0.093645334, -0.068808675, 0.17389251, -0.24..."
138698,4,nosleep,abd3v8,1546302452,"I work as a paranormal specialist, these are s...",[Part 1](https://www.reddit.com/r/nosleep/comm...,185,"[-37.412224, -31.799099, 52.03939, -53.81167, ...","[-0.20222823, -0.17188703, 0.281294, -0.290873..."
138699,5,nosleep,abdduv,1546304387,They strike at midnight,Little information you guys should have before...,61,"[-9.491199, -1.2185694, 25.51978, -25.440092, ...","[-0.15559343, -0.019976547, 0.41835704, -0.417..."
138700,6,nosleep,abde81,1546304458,"""It's eyes and mouth took up most of its face""","Our minds love to play tricks on us, in fact a...",42,"[0.6389348, -2.2228947, 18.655128, -17.523226,...","[0.015212733, -0.052926064, 0.44416973, -0.417..."
...,...,...,...,...,...,...,...,...,...
174073,91,nosleep,ei9t2d,1577831046,Transcription from an old casette.,\n\nThis is a testimony I transcribed from a...,45,"[6.04609, -4.7146564, 7.449227, -13.029264, -7...","[0.13435756, -0.10477014, 0.16553837, -0.28953..."
174074,95,nosleep,eia866,1577833074,The first and last time I performed a brain ti...,Jim A.\n\n\-----------------------------------...,59,"[7.911386, -11.531252, 26.039703, -11.786053, ...","[0.13409129, -0.19544494, 0.4413509, -0.199763..."
174075,96,nosleep,eiaati,1577833428,I found something in the worm box,About a week ago I purchased one of those smal...,60,"[3.2293615, -1.678937, 7.457329, -4.191322, -5...","[0.053822692, -0.027982283, 0.12428881, -0.069..."
174076,98,nosleep,eiah2c,1577834259,I think I moved to the wrong area...(part 5),[Part 1](https://www.reddit.com/r/nosleep/comm...,125,"[-9.9853, -15.488511, 27.843788, -26.632832, -...","[-0.0798824, -0.12390809, 0.2227503, -0.213062..."


In [4]:
from creepy_module import clean_data, clean_comments

import os
import pandas as pd
import pickle

PATH = 'Download/Raw Data'
OUTPUT_PATH = 'Download/pickles/'

for subreddit in os.listdir(PATH):
        
    try:
        os.makedirs(f"{OUTPUT_PATH + subreddit}")
    except FileExistsError:
        pass
    
    print(f'''
    -----------
    {subreddit}
    -----------
    ''')
    
    subreddit_df_li = []
    
    if subreddit.startswith('Spanish'):
        for file in os.listdir(os.path.join(PATH, subreddit, subreddit)):
            if file.startswith('RS_'):
                print(os.path.join(PATH, subreddit, subreddit, file))
                # Data
                try:
                    subreddit_df_li.append(clean_data(os.path.join(PATH, subreddit, subreddit, file), lang='es'))
                except:
                    print('ERROR!!: ', os.path.join(PATH, subreddit, subreddit, file))
                
        subreddit_df = pd.concat(subreddit_df_li)
    else:
        for file in os.listdir(os.path.join(PATH, subreddit, subreddit)):
            if file.startswith('RS_'):
                # Data
                try:
                    subreddit_df_li.append(clean_data(os.path.join(PATH, subreddit, subreddit, file)))
                except:
                    print('ERROR!!: ', os.path.join(PATH, subreddit, subreddit, file))
                
        subreddit_df = pd.concat(subreddit_df_li)
        
        
    pickle.dump(subreddit_df, open(os.path.join(OUTPUT_PATH, f"{subreddit}.pickle"), 'wb'))


    -----------
    Confession
    -----------
    
Reading data...
Done
Removing nonexisting rows...
Done
Removing links...
ERROR!!:  Download/Raw Data/Confession/Confession/RS_confession.csv


ValueError: No objects to concatenate